<a href="https://colab.research.google.com/github/pedromonteirocs/Learn_Machine_Learning_in_3_Months/blob/master/TestesComSpotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Spotipy

In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

In [3]:
# client_id= "eeef52fb50cb44dd82a0887f30d96f2f"
# client_secret= "c65713bc0ccf4985b998e53b97de6ff4"
# redirect_uri='Exemplo: http://google.com/'
# username='pedromonteirocs'
# playlist = 'ID_DA_SUA_PLAYLIST'

In [4]:
#Input your Spotify credentials below

CLIENT_ID = "eeef52fb50cb44dd82a0887f30d96f2f"
CLIENT_SECRET =  "c65713bc0ccf4985b998e53b97de6ff4"
username = 'pedromonteirocs'
#username = 
market = ['BR']
redirect_uri='http://localhost:8080/callback/'


In [5]:
credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)
scope = 'user-library-read' #scope = 'user-library-read playlist-modify-public playlist-read-private' #This is an alternate scope...

#token = util.prompt_for_user_token(username, scope, CLIENT_ID, CLIENT_SECRET, redirect_uri) # Use this if the token above doesn't work.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  """


In [6]:
#Enter your own playlist for custom results

PLAYLIST_ID = '37i9dQZF1EpvQqCl4eXhB3'
#PLAYLIST_ID = '2v9D2pznYNnAoxgUyneIGN' #Juans playlist

def get_playlist_tracks(username,playlist_id):
    results = spotify.user_playlist_tracks(username,playlist_id,limit=100, offset=0)
    tracks = results['items']
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])
    return tracks


#playlist_tracks=get_playlist_tracks(CLIENT_ID,PLAYLIST_ID)

#playlist do benei
playlist_tracks=get_playlist_tracks(CLIENT_ID,'21sYYcgcMO3do3TcDEBy5z')


#playlist_tracks = spotify.user_playlist_tracks(CLIENT_ID, PLAYLIST_ID, fields='items,uri,name,id,total')

In [7]:
len(playlist_tracks)

57

Nomes das Músicas

In [8]:
teste=playlist_tracks["items"][0]['track']['name']
playlist_tracks['items'][0]['track']['artists'][0]['name']

TypeError: ignored

IDs dos artistas e Músicas da Playlist e Nomes das Músicas

In [9]:
songIDs, artistIDs, nomeMusica, artistaNome = [], [],[],[]

#Find the name, artist, and ID for each song in the playlist
for i in range(len(playlist_tracks)):
  artistIDs.append(playlist_tracks[i]['track']['artists'][0]['id']) #Append artist IDs to a list
  songIDs.append(playlist_tracks[i]['track']['id']) #Append song IDs to a list
  nomeMusica.append(playlist_tracks[i]['track']['name']) #append nome da musica
  artistaNome.append(playlist_tracks[i]['track']['artists'][0]['name']) #append nome do artista
  
print(songIDs)
print(artistIDs)
print(artistaNome)
print(nomeMusica)

['36P6ptTyxOggYgR2aoZiAl', '71FrGl0XcpL3VId65tB5mg', '4JPSfCXQe6kDliMIJTqISW', '4k20Ie47EcWtS4wMBn8w8S', '2AY8dKUoSLHzpIePuQOzU8', '4zGtfOglDBfVUypabBwIjQ', '7Ac3BmqTQoLdAt7HtZyfgN', '4xWzZmX4K1yyrdtRfbUvjt', '14kRWkCWBYfQqILflEcpSJ', '3Hb9kUdm4yf839Fle4RIdT', '7rLegaz7zKB6EtDNzcslKE', '4lKuzrdZrJuCAedbIslF3A', '7rvW49PvaFKGyfeoxay9Go', '4HUZBG98TYbxSR9V1V2DWS', '7FiIQXpIeLo3ITfGGpb8PB', '5dKesZwp6deuhEeW8F1UEi', '6V43u5SCjkfMpxMAXRPifr', '0ZHK2kaWTvSCRWkLflRFgF', '63SHcJzckFykS8faA5zojY', '1gCEwGf1XDVsGtUbNcgnxw', '5F8ffc8KWKNawllr5WsW0r', '6FWMkcaZyKS8I60zR6ro7h', '12v6LfkX9YIR3uLefIAAYZ', '6AaoEX192rJ6o3UFwG43sV', '3j7FDNUqIMWKaji6PK5SyF', '3unTxk7OFAo1mIyL4si9zj', '2loApP0kPaim0BTDVP9F26', '0AGS6ZRgzobrazmCi6pYMe', '2bPtwnrpFNEe8N7Q85kLHw', '1OUPXna2MCgAt3VNmXJBtg', '2RMOwSvOGQyy985Vx4VSGQ', '2H8ETIUEoN9YMf56X66bxd', '1OtZIaJBXo3Pe6nfaMsjKT', '70rCH90CeRGzCrVr4fSbqg', '0nkDm27DGppCSk8dnGtvPa', '2WlZuBDgLfT7Kc0admhFdg', '6cCmWeefBjzsgDRceD4AXw', '7LPR6AdkOQR9WpEixCGNCk', '7pDcS0EVMJ

Features baseadas nas músicas

In [10]:
features_list = []
features=spotify.audio_features(songIDs)
for ident in songIDs: #for each song in the user's playlist get the features
  features=spotify.audio_features(ident)[0]
  features_list.append(([features['energy'], features['liveness'], features['tempo'], features['speechiness'], features['acousticness'], features['instrumentalness'],
                          features['time_signature'], features['danceability'], features['key'], features['duration_ms'], features['loudness'], features['valence'], 
                          features['mode']]))

In [11]:
#Read features into a df
df = pd.DataFrame(features_list, columns=['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'key', 'duration_ms', 
                                          'loudness', 'valence', 'mode'])

print(df.head())

df.shape

   energy  liveness    tempo  speechiness  ...  duration_ms  loudness  valence  mode
0   0.874    0.0821  164.810       0.1280  ...       158662    -3.162    0.829     1
1   0.937    0.6840  168.097       0.0861  ...       166838    -2.118    0.938     1
2   0.680    0.9120  188.162       0.2920  ...       170507    -7.255    0.643     0
3   0.864    0.9320  134.992       0.0517  ...       157494    -4.281    0.701     1
4   0.866    0.0562  118.011       0.2590  ...       174515    -1.568    0.879     1

[5 rows x 13 columns]


(57, 13)

In [ ]:
# features_list = []

# for i in range(len(songIDs)): #for each song in the user's playlist
#   thing = spotify.audio_features(songIDs)
  
#   for features in thing: #save each feature in the audio features json dict
#     features_list.append([features['energy'], features['liveness'], features['tempo'], features['speechiness'], features['acousticness'], features['instrumentalness'],
#                           features['time_signature'], features['danceability'], features['key'], features['duration_ms'], features['loudness'], features['valence'], 
#                           features['mode']])

# #Read features into a df
# df = pd.DataFrame(features_list, columns=['energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'key', 'duration_ms', 
#                                           'loudness', 'valence', 'mode'])

# print(df.head())



In [ ]:
df.shape

In [ ]:
X=np.array(df)

labels = np.array(nomeMusica)

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

pred_classes = kmeans.predict(X)

print(pred_classes)

Criar Clusters

In [ ]:
N_CLUSTERS=4
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0).fit(X)

pred_classes = kmeans.predict(X)


In [ ]:
X.shape
labels.size
range(len(songIDs))
for cluster in range(N_CLUSTERS):
    print('cluster: ', cluster)
    print(labels[np.where(pred_classes == cluster)])